In [1]:
import os
import pickle

In [2]:
from isanlp import PipelineCommon
from isanlp.processor_remote import ProcessorRemote
from isanlp.ru.processor_mystem import ProcessorMystem
from isanlp.ru.converter_mystem_to_ud import ConverterMystemToUd

In [3]:
def segmentation(tree, text, segments):
    if tree.left:
        segmentation(tree.left, text, segments)
        segmentation(tree.right, text, segments)
    else:
        segments.append(text[tree.start:tree.end])

In [4]:
ppl = PipelineCommon([
    (ProcessorRemote('localhost', 3334, '0'),
     ['text'],
     {'sentences': 'sentences',
      'tokens': 'tokens',
      'lemma': 'lemma',
      'syntax_dep_tree': 'syntax_dep_tree',
      'postag': 'ud_postag'}),
    (ProcessorMystem(delay_init=False),
 ['tokens', 'sentences'],
     {'postag': 'postag'}),
    (ConverterMystemToUd(),
     ['postag'],
     {'morph': 'morph',
      'postag': 'postag'}),
    (ProcessorRemote('localhost', 3335, 'default'),
     ['text', 'tokens', 'sentences', 'postag', 'morph', 'lemma', 'syntax_dep_tree'],
     {'rst': 'rst'})
])

In [5]:
texts = []
names = os.listdir('../rus-human-tr')
for name in names:
    if name != 'segmented':
        with open('../rus-human-tr' + '/' + name, 'r', encoding='utf-8') as f:
            text = f.read()
            text = text.split('\n')
            text = ' '.join(text)
            texts.append(text)

In [6]:
texts[0]

'Да, очень раздражает и утомляет все время сортировать мусор правильно. На кухне воняют три разных мешка для мусора. Их нужно отсортировать в разные мусорные баки. Но все же Германия производит слишком много мусора и теряется слишком много ресурсов, когда сжигается то, что на самом деле следует разделять и перерабатывать. Мы, берлинцы, должны воспользоваться шансом и стать первопроходцами в разделении отходов!'

In [7]:
def extr_pairs(tree, text):
    pp = []
    if tree.left:
        pp.append([text[tree.left.start:tree.left.end],
                   text[tree.right.start:tree.right.end], 
                   tree.relation, tree.nuclearity])
        pp += extr_pairs(tree.left, text)
        pp += extr_pairs(tree.right, text)
    return pp

In [ ]:
%%time

for i in range(len(texts)):
    text = texts[i]
    name = names[i]
    if name.split('.')[0] in ['micro_b006', 'micro_b007', 'micro_b029', 'micro_b034', 'micro_b038']:
        result = ppl(text)
        tree = result['rst'][0]
        tree.to_rs3('file.rs3')
        text = result['text']
        edus = extr_pairs(tree, text)
        print(str(i)) #+ '\n')
        if False:
            with open('../русский авторазметчик эде' + '/' + 'ar_' + name.split('.')[0] + '.pickle', 'wb') as f:
                pickle.dump(edus, f)
            with open('../русский авторазметчик эде' + '/' + 'ar_tree_' + name.split('.')[0] + '.pickle', 'wb') as f:
                pickle.dump(tree, f)    
        with open('../русский авторазметчик эде' + '/' + 'ar_result_' + name.split('.')[0] + '.pickle', 'wb') as f:
            pickle.dump(result, f)
#    print(extr_pairs(result['rst'][0], result['text']))

In [ ]:
stop

## Дальше идёт черновик

In [ ]:
%%time

for i in range(10):#len(texts)):
    text = texts[i]
    name = names[i]
    result = ppl(text)
    tree = result['rst'][0]
    text = result['text']
    segments = []
    segmentation(tree, text, segments)
    print(i + '\n')
    print(tree)
    with open('../rus-human-tr/segmented' + '/' + 's_' + name, 'w') as f:
        f.write('\n'.join(segments))

In [ ]:
with open('../rus-human-tr/segmented\test.txt', 'w') as f:
        f.write('\n'.join(segments))

In [ ]:
texts = []
names = os.listdir('../rus-human-tr/segmented')
for name in names:
    with open('../rus-human-tr/segmented' + '/' + name, 'r') as f:
        text = f.read()
        text = text.split('\n')
        for edu in text:
            texts.append(edu)
        texts.append('\n')

In [ ]:
import pandas as pd

In [ ]:
df_test = pd.DataFrame(texts)

In [ ]:
df_test.to_excel('aaaaaa.xlsx')

In [ ]:
import copy
import os
import pickle

import pandas as pd

from pymystem3 import Mystem

In [ ]:
texts = {}
trees = {}
results = {}
names = os.listdir(r'../русский авторазметчик эде')
for name in names:
    with open(r'../русский авторазметчик эде' + '/' + name, 'br') as f:
            obj = pickle.load(f)
            if 'tree' in name:
                trees[name] = obj
            else:
                if 'result' in name:
                    results[name] = obj
                else:
                    texts[name] = obj

токены: все у которых бегин правее, а енд левее. У каждого токена лемма из словаря токены — леммы, словарь один для текста

In [ ]:
def lemmatize(tree, tokens, lemmas):
    words = []
    for token in tokens:
        if tree.start <= token.begin and token.end <= tree.end:
            words.append(lemmas[token])
    return word

In [ ]:
trees['ar_tree_micro_k031.pickle'].right.start

In [ ]:
results['ar_result_micro_k031.pickle']['sentences'][1]

In [ ]:
results['ar_result_micro_k031.pickle']